Copyright © 2021-2022 HQS Quantum Simulations GmbH. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except
in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.

# Example: Half Adder
In this example we write a quantum algorithm to perform an add operation between two qubits and store the result in two qubits that act as output register.

The truth table of the circuit is

| in q1 | in q2 | out q4 | out q3 | out  (integer) |
|-------|-------|--------|--------|----------------|
| 0     | 0     | 0      | 0      | 0              |
| 0     | 1     | 0      | 1      | 1              |
| 1     | 0     | 0      | 1      | 1              |
| 1     | 1     | 1      | 0      | 2              |

Please note that for simplicity we implement a half adder that picks up an additional phase. This circuit only works as a half-adder if the result is measured immediately. A half-adder implementation without an extra phase would require a decomposition of the Toffoli gate which requires more than three CNOT gates.

The circuit that we will code to implement a half adder is as follows:

![half adder](half_adder.png)

The two `X` operations at the beginning are used to generate the input, in this case both input qubits are set to `1`.

In [ ]:
# A small helper routine for nice output
def get_binary(x: int, size: int) -> str:
    """Return a string representation of int x with size positions."""
    return format(x, 'b').zfill(size)

We import required classes and modules from qoqo. Circuit is the main class to represent quantum circuits. The operations module contains one- and two-qubit operations such as Hadamard or CNOT.

In [ ]:
from qoqo import Circuit
from qoqo import operations as ops
from pprint import pprint as pp
import math

We define a method to generate the main block of the half-adder circuit.

In [ ]:
def half_adder_main() -> Circuit():
    half_adder = Circuit(4)
    ####################
    # Least relevant bit
    half_adder += ops.CNOT(control=0, target=2)
    half_adder += ops.CNOT(control=1, target=2)
    ###################
    # Most relevant bit
    # Controlled H
    half_adder += ops.RotateY(3, math.pi / 4)
    half_adder += ops.CNOT(0, 3)
    half_adder += ops.RotateY(3, -math.pi / 4)
    # Controlled Z
    half_adder += ops.Hadamard(3)
    half_adder += ops.CNOT(1, 3)
    half_adder += ops.Hadamard(3)
    # Controlled H
    half_adder += ops.RotateY(3, math.pi / 4)
    half_adder += ops.CNOT(0, 3)
    half_adder += ops.RotateY(3, -math.pi / 4)
    return half_adder

Let's add everything together. We add a complex classical register to store the state vector of the qubits after our calculation. Other types of registers available in qoqo are `DefinitionBit` for bit registers used to store actual measurement results of a quantum computer and `DefinitionFloat` to store real valued results.

We use the `GetStateVector` Pragma operation. A Pragma operation is information for the compiler / qoqo only and will not be sent to the actual quantum computer. We will encounter other important Pragmas later on. The `GetStateVector` Pragma operation obtains the state vector of the qubits and stores it in the defined output register ('ro'). It accepts an additional measurement circuit that would be added to the circuit before measuring. The state vector can only be obtained in simulations on conventional computers, never from the real device.

In [ ]:
half_adder = Circuit(4)
half_adder += ops.DefinitionComplex('ro', 2**4, is_output=True)
# Init
half_adder += ops.PauliX(0)
half_adder += ops.PauliX(1)
#main
half_adder += half_adder_main()
# Measure
half_adder += ops.PragmaGetStateVector('ro', Circuit())

In [ ]:
print(half_adder)
# from qoqo_qasm import qasm_call_circuit
# qasm_call_circuit(half_adder_main())

We simulate the half adder using `qoqo_quest`. Running the circuit in the backend returns a tuple with entries for all registers of the three different types.

We print the complex coefficients (amplitudes) of the quantum state. The input qubits are the two bits on the right side (as they are qubits 0 and 1) while the output is stored in the most relevant bits (the two right bits). As expected, the state `|1011>` is populated while all other states are empty.

In [ ]:
from qoqo_quest import Backend
backend = Backend(4)
(
    res_bit_registers,
    res_float_registers,
    res_complex_registers
) = backend.run_circuit(half_adder)
pp({
    get_binary(x, 4): value.real for x, value in enumerate(
        res_complex_registers['ro'][0]
    )
})

## Simulating an experiment

How would the result that we would get from a real quantum computer look? Let's try it out!

We define a number of measurements, i.e., repetitions and measurements of the circuit and add a bit register to store the measured values for each run.

We add `MeasureQubit` operations for the two output qubits.
To make this more interesting we initialize the input qubits in a superposition of all possible states using Hadamard gates `H`.

In [ ]:
number_measurements = 1000
half_adder = Circuit(4)
half_adder += ops.DefinitionBit('ro', 2, is_output=True)
# Input
half_adder += ops.Hadamard(0)
half_adder += ops.Hadamard(1)
# Main
half_adder += half_adder_main()
# Measure
half_adder += ops.MeasureQubit(2, 'ro', readout_index=0)
half_adder += ops.MeasureQubit(3, 'ro', readout_index=1)
half_adder += ops.PragmaSetNumberOfMeasurements(number_measurements, 'ro')


In [ ]:
(
    res_bit_registers,
    res_float_registers,
    res_complex_registers
) = backend.run_circuit(half_adder)
# transform measured bitstring into integers
results = [int(f"{int(x[1])}{int(x[0])}", base=2) for x in res_bit_registers['ro']]
# pp(results)

We plot the histogram of the measured bitstring interpreted as integer. We expect 25% 0, 50% 1 and 25% 2.

In [ ]:
from matplotlib import pyplot as plt
plt.hist(results, bins=[-0.5, 0.5, 1.5, 2.5])